In [9]:
import finlab
from finlab import data
import pandas as pd
import datetime as dt
import numpy as np
import math
from dotenv import load_dotenv
import os
import time
from finlab.backtest import sim
import scipy.stats as stats

def config():
    load_dotenv()


    
config()

finlab.login(str(os.getenv('FINLAB_API')))

輸入成功!


In [10]:
adj_price = data.get('etl:adj_close')

Daily usage: 320.5 / 500 MB - etl:adj_close


In [6]:

adj_price = data.get('etl:adj_close')
inventory = data.get('inventory')

Daily usage: 288.4 / 500 MB - etl:adj_close


ConnectionError: ('Connection aborted.', OSError(22, 'Invalid argument'))

In [44]:
inventory.info()

<class 'finlab.dataframe.FinlabDataFrame'>
RangeIndex: 8899600 entries, 0 to 8899599
Data columns (total 6 columns):
 #   Column    Dtype         
---  ------    -----         
 0   stock_id  object        
 1   date      datetime64[ns]
 2   持股分級      object        
 3   人數        int64         
 4   持有股數      int64         
 5   占集保庫存數比例  float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 407.4+ MB


In [59]:
"""
持股分級:
{1:'1-999',2:'1,000-5,000',3:'5,001-10,000',
4:'10,001-15,000',5:'15,001-20,000',6:'20,001-30,000',
7:'30,001-40,000',8:'40,001-50,000',9:'50,001-100,000',
10:'100,001-200,000',11:'200,001-400,000',12:'400,001-600,000',
13:'600,001-800,000',14:'800,001-1,000,000',15:'1,000,001以上',17:'合計'}

"""

inventory
modified_inventory = inventory.set_index(['date', 'stock_id'])
modified_inventory['持股分級']= modified_inventory['持股分級'].astype(int)
modified_inventory =modified_inventory[(modified_inventory['持股分級'] > 13) & (modified_inventory['持股分級'] < 17)]
#using square to increase the significance of 持股分級
#modified_inventory['持股分級'] = modified_inventory['持股分級'] **2

#using times 100 (X100) to increase the significance of 持股分級
#modified_inventory['持股分級'] = modified_inventory['持股分級'] *100
modified_inventory.info()

<class 'finlab.dataframe.FinlabDataFrame'>
MultiIndex: 1112450 entries, (Timestamp('2016-11-04 00:00:00'), '0050') to (Timestamp('2020-12-31 00:00:00'), '9965')
Data columns (total 4 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   持股分級      1112450 non-null  int64  
 1   人數        1112450 non-null  int64  
 2   持有股數      1112450 non-null  int64  
 3   占集保庫存數比例  1112450 non-null  float64
dtypes: float64(1), int64(3)
memory usage: 38.4+ MB


In [61]:
modified_inventory

持股分級  人數       持有股數  占集保庫存數比例
date       stock_id                               
2016-11-04 0050        14   3    2782000      0.47
           0050        15  46  423873000     71.66
2016-11-11 0050        14   2    1925000      0.32
           0050        15  46  420532000     71.09
2016-11-18 0050        14   4    3637000      0.59
...                   ...  ..        ...       ...
2020-12-18 9965        15   0          0      0.00
2020-12-25 9965        14   0          0      0.00
           9965        15   0          0      0.00
2020-12-31 9965        14   0          0      0.00
           9965        15   0          0      0.00

[1112450 rows x 4 columns]

In [60]:
# Define thresholds for identifying large holdings
shares_threshold = 10000000  # Example threshold for '持有股數'
proportion_threshold = 10  # Example threshold for '占集保庫存數比例' in percentage
holding_level_threshold = 13
# Filter for rows where either '持有股數' or '占集保庫存數比例' exceed the thresholds
big_whales = modified_inventory[(modified_inventory['持有股數'] >= shares_threshold) |
                           (modified_inventory['占集保庫存數比例'] >= proportion_threshold) |
                           (modified_inventory['持股分級'] > holding_level_threshold)]
big_whales

持股分級  人數       持有股數  占集保庫存數比例
date       stock_id                               
2016-11-04 0050        14   3    2782000      0.47
           0050        15  46  423873000     71.66
2016-11-11 0050        14   2    1925000      0.32
           0050        15  46  420532000     71.09
2016-11-18 0050        14   4    3637000      0.59
...                   ...  ..        ...       ...
2020-12-18 9965        15   0          0      0.00
2020-12-25 9965        14   0          0      0.00
           9965        15   0          0      0.00
2020-12-31 9965        14   0          0      0.00
           9965        15   0          0      0.00

[1112450 rows x 4 columns]